In [1]:
import pandas as pd

df = pd.read_csv('Processed_Headings/normalized_section_data 2.csv')
# df['sec-title'] = df.apply(lambda row: row['xmlPath'].rsplit(' > ', 1)[-1].strip(), axis=1)
# df.to_csv('Processed_Headings/combined.csv')
df.head()

,PMCID,PMID,sec-type,sec-title,sec-norm
0,PMC5427117,28337662,NaN,Introduction,background
1,PMC5427117,28337662,NaN,Methods,methods
2,PMC5427117,28337662,NaN,Results,results
3,PMC5427117,28337662,NaN,Discussion,conclusions
4,PMC5427117,28337662,NaN,Electronic supplementary material,background


In [1]:
import pandas as pd

df_test = pd.read_csv("data/test_annotated.csv")
df_test

,PMCID,PMID,sec-title,sec-norm
0,PMC11184573,38888203,"['INTRODUCTION', 'MATERIALS AND METHODS', 'DIS...","['background', 'methods', 'conclusions', 'conc..."
1,PMC11183239,38886373,"['Introduction', 'Subjects and methods', 'Resu...","['background', 'methods', 'results', 'conclusi..."
2,PMC11187291,38903966,"['Introduction', 'Methods', 'Results', 'Discus...","['background', 'methods', 'results', 'conclusi..."
3,PMC11177991,38883749,"['INTRODUCTION', 'MATERIALS AND METHODS', 'RES...","['background', 'methods', 'results', 'conclusi..."
4,PMC11187304,38903638,"['Introduction', 'Methods', 'Case reports', 'D...","['background', 'methods', 'methods', 'conclusi..."
...,...,...,...,...
155,PMC11181322,38771959,"['Introduction', 'Results and Discussion', 'Co...","['background', 'conclusions', 'conclusions', '..."
156,PMC11186409,38899065,"['Introduction', 'Materials and methods', 'Res...","['background', 'methods', 'results', 'conclusi..."
157,PMC11185683,38895272,"['Introduction', 'Results', 'Discussion', 'Mat...","['background', 'results', 'conclusions', 'meth..."
158,PMC11161898,38854388,"['METHODS', 'RESULTS', 'DISCUSSION', 'Notes']","['methods', 'results', 'conclusions', 'backgro..."


In [2]:
import ast
df_test['sec-title'] = df_test['sec-title'].apply(ast.literal_eval)
df_test['sec-title']

0      [INTRODUCTION, MATERIALS AND METHODS, DISCUSSI...
1      [Introduction, Subjects and methods, Results, ...
2      [Introduction, Methods, Results, Discussion, C...
3      [INTRODUCTION, MATERIALS AND METHODS, RESULTS,...
4      [Introduction, Methods, Case reports, Discussi...
                             ...                        
155    [Introduction, Results and Discussion, Conclus...
156    [Introduction, Materials and methods, Results,...
157    [Introduction, Results, Discussion, Materials ...
158                [METHODS, RESULTS, DISCUSSION, Notes]
159    [1 Introduction, 2 Methods, 3 Results, 4 Discu...
Name: sec-title, Length: 160, dtype: object

In [3]:
import numpy as np
def eval_with_nan(x):
    if isinstance(x, list):
        return [np.nan if (isinstance(i, float) and np.isnan(i)) else i for i in x]
    try:
        # 這邊直接 eval 而且給定 locals
        val = eval(x, {"nan": np.nan, "NaN": np.nan, "np": np})
        if isinstance(val, list):
            return [np.nan if (isinstance(i, float) and np.isnan(i)) else i for i in val]
        else:
            return val
    except Exception:
        return x  # 如果 eval 還失敗就原樣保留


In [4]:
df_test['sec-norm'] = df_test['sec-norm'].apply(eval_with_nan)
df_test['sec-norm']

0      [background, methods, conclusions, conclusions...
1      [background, methods, results, conclusions, co...
2      [background, methods, results, conclusions, co...
3      [background, methods, results, conclusions, co...
4      [background, methods, methods, conclusions, co...
                             ...                        
155    [background, conclusions, conclusions, methods...
156    [background, methods, results, conclusions, ba...
157          [background, results, conclusions, methods]
158          [methods, results, conclusions, background]
159    [background, methods, results, conclusions, ba...
Name: sec-norm, Length: 160, dtype: object

In [5]:
type(df_test['sec-norm'][0])

list

In [6]:
from itertools import chain
texts = list(chain.from_iterable(df_test["sec-title"]))
texts

['INTRODUCTION',
 'MATERIALS AND METHODS',
 'DISCUSSION',
 'CONCLUSION',
 'AUTHOR CONTRIBUTIONS',
 'FUNDING INFORMATION',
 'CONFLICT OF INTEREST STATEMENT',
 'ETHICS STATEMENT',
 'Supporting information',
 'DATA AVAILABILITY STATEMENT',
 'Introduction',
 'Subjects and methods',
 'Results',
 'Discussion',
 'Conclusion',
 'Supplementary Information',
 'Introduction',
 'Methods',
 'Results',
 'Discussion',
 'Conclusion',
 'Data availability statement',
 'Ethics statement',
 'Author contributions',
 'Conflict of interest',
 "Publisher's note",
 'Supplementary material',
 'INTRODUCTION',
 'MATERIALS AND METHODS',
 'RESULTS',
 'DISCUSSION',
 'Conclusions',
 'Supplementary Material',
 'Availability of data and materials',
 'Introduction',
 'Methods',
 'Case reports',
 'Discussion',
 'Conclusion',
 'Data availability statement',
 'Ethics statement',
 'Author contributions',
 'Conflict of interest',
 'Publisher‚Äôs note',
 'Introduction',
 'Materials and methods',
 'Results',
 'Discussion',
 'C

In [7]:

from sentence_transformers import SentenceTransformer, InputExample, losses
fine_tuned_model = SentenceTransformer("models/fine_tuned_sentence_bert_model_ContrastiveLoss_test_lower")

# 測試新模型
embeddings = fine_tuned_model.encode([text.lower() for text in texts])

In [8]:
import numpy as np

np.save("comparison/embeddings_test_lower0.npy", embeddings)

In [9]:
norms = list(chain.from_iterable(df_test["sec-norm"]))
norms

['background',
 'methods',
 'conclusions',
 'conclusions',
 'background',
 'background',
 'background',
 'background',
 'background',
 'background',
 'background',
 'methods',
 'results',
 'conclusions',
 'conclusions',
 'background',
 'background',
 'methods',
 'results',
 'conclusions',
 'conclusions',
 'background',
 'background',
 'background',
 'background',
 'background',
 'background',
 'background',
 'methods',
 'results',
 'conclusions',
 'conclusions',
 'background',
 'background',
 'background',
 'methods',
 'methods',
 'conclusions',
 'conclusions',
 'background',
 'background',
 'background',
 'background',
 'background',
 'background',
 'methods',
 'results',
 'conclusions',
 'conclusions',
 'background',
 'methods',
 'results',
 'conclusions',
 'conclusions',
 'background',
 'background',
 'background',
 'background',
 'background',
 'results',
 'conclusions',
 'methods',
 'background',
 'background',
 'background',
 'background',
 'methods',
 'results',
 'conclusions',


In [10]:
label2id = {
     "methods" : 0,
     "background" : 1,
     "results" : 2,
     "conclusions" : 3, 
     "objective": 4
}
labels = np.array([label2id[label] if label in label2id else -1 for label in norms])
np.save("comparison/labels_lower0.npy", labels)